In [98]:
from sklearn.tree import *
from sklearn.metrics import classification_report
import numpy as np
import joblib
import os
from sklearn.naive_bayes import *
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from deap import *

In [ ]:
data = np.load("C:\\Users\\Alan Phan\\Desktop\\Bach Khoa Studies\\HK242\\Machine Learning\\Assignments\\AI-MedMNIST-Classification\\AI-MedMNIST-Classification\\data\\preprocessed\\train_data.npz")
x_train, y_train = data["x_train"], data["y_train"].ravel()

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

data = np.load("C:\\Users\\Alan Phan\\Desktop\\Bach Khoa Studies\\HK242\\Machine Learning\\Assignments\\AI-MedMNIST-Classification\\AI-MedMNIST-Classification\\data\\preprocessed\\test_data.npz")
x_test, y_test = data["x_test"], data["y_test"].ravel()


In [ ]:
x_train = x_train.reshape(x_train.shape[0], -1)
x_test = x_test.reshape(x_test.shape[0], -1)

pca = PCA(n_components=10)
x_train_pca = pca.fit_transform(x_train)
x_test_pca = pca.transform(x_test)

x_train_pca, x_val_pca, y_train, y_val = train_test_split(x_train_pca, y_train, test_size=0.2, random_state=42)


In [101]:
from deap import base, creator, tools, algorithms
import random
from sklearn.metrics import accuracy_score

def fitness_function(params):
    var_smoothing = 10 ** params[0]
    model = GaussianNB(var_smoothing=var_smoothing)
    model.fit(x_train_pca, y_train)
    
    y_pred = model.predict(x_val_pca)
    accuracy = accuracy_score(y_val, y_pred)
    
    return (accuracy,)


In [102]:
# GA Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))  
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 0)  
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", fitness_function)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)

POP_SIZE = 10
NGEN = 50
MUTPB = 0.2
CXPB = 0.5

population = toolbox.population(n=POP_SIZE)
algorithms.eaSimple(population, toolbox, cxpb=CXPB, mutpb=MUTPB, ngen=NGEN, 
                    stats=None, halloffame=None, verbose=True)

best_ind = tools.selBest(population, k=1)[0]
best_var_smoothing = 10 ** best_ind[0]
print(f"Best var_smoothing: {best_var_smoothing}")


c:\Users\Alan Phan\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\Alan Phan\AppData\Local\Programs\Python\Python312\Lib\site-packages\deap\creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


gen	nevals
0  	10    
1  	2     
2  	6     
3  	2     
4  	4     
5  	5     
6  	4     
7  	7     
8  	6     
9  	9     
10 	7     
11 	8     
12 	4     
13 	5     
14 	4     
15 	7     
16 	6     
17 	10    
18 	6     
19 	7     
20 	8     
21 	5     
22 	4     
23 	6     
24 	6     
25 	4     
26 	8     
27 	2     
28 	4     
29 	6     
30 	5     
31 	4     
32 	5     
33 	5     
34 	6     
35 	6     
36 	4     
37 	7     
38 	8     
39 	7     
40 	6     
41 	4     
42 	5     
43 	4     
44 	5     
45 	8     
46 	8     
47 	7     
48 	8     
49 	8     
50 	6     
Best var_smoothing: 1.775960186321084e-05


In [103]:
final_model = GaussianNB(var_smoothing=best_var_smoothing)
final_model.fit(x_train_pca, y_train)

y_pred = final_model.predict(x_test_pca)

# Evaluate performance
print(classification_report(y_test, y_pred))

# save model
model_save_path = os.path.join("models", "trained", "naive_bayes_with_ga_best_grid.pkl")

# Ensure the directory exists
os.makedirs(os.path.dirname(model_save_path), exist_ok=True)

# Save the model
joblib.dump(final_model, model_save_path)
print(f"Model saved successfully at: {model_save_path}")


              precision    recall  f1-score   support

           0       0.89      0.86      0.88      1338
           1       0.55      0.99      0.71       847
           2       0.43      0.54      0.48       339
           3       0.70      0.57      0.63       634
           4       0.54      0.23      0.32      1035
           5       0.59      0.08      0.14       592
           6       0.42      0.22      0.29       741
           7       0.25      0.65      0.37       421
           8       0.58      0.67      0.62      1233

    accuracy                           0.57      7180
   macro avg       0.55      0.54      0.49      7180
weighted avg       0.60      0.57      0.54      7180

Model saved successfully at: models\trained\naive_bayes_with_ga_best_grid.pkl
